<a href="https://www.kaggle.com/code/derekxue/petfacialexpression?scriptVersionId=141196211" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pets-facial-expression-dataset/Other/aug-88-54.jpg
/kaggle/input/pets-facial-expression-dataset/Other/45.jpg
/kaggle/input/pets-facial-expression-dataset/Other/56.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-141-02.jpg
/kaggle/input/pets-facial-expression-dataset/Other/20.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-37-49.jpg
/kaggle/input/pets-facial-expression-dataset/Other/58.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-168-37.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-80-43.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-183-55.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-114-30.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-152-19.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-180-52.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-195-12.jpg
/kaggle/input/pets-facial-expression-dataset/Other/aug-123-39.jpg
/kaggle/input/pets-facial-expression-dataset/

In [2]:
!pip install fastai -Uqq

In [3]:
import random, os
import numpy as np

from fastai.data.all import *
from fastai.vision.all import *
from fastai.tabular.all import *

import warnings
# Ignore warning messages
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Bring in datasets. 

In [4]:
face_source = Path('/kaggle/input/pets-facial-expression-dataset')

In [5]:
face_source.ls()

(#5) [Path('/kaggle/input/pets-facial-expression-dataset/Other'),Path('/kaggle/input/pets-facial-expression-dataset/Angry'),Path('/kaggle/input/pets-facial-expression-dataset/Sad'),Path('/kaggle/input/pets-facial-expression-dataset/Master Folder'),Path('/kaggle/input/pets-facial-expression-dataset/happy')]

In [6]:
# build an image file meta data frame. 
meta_df = pd.DataFrame({
    'image':(
        *get_image_files(face_source),
    )
})

meta_df.head()

,image
0,/kaggle/input/pets-facial-expression-dataset/Other/aug-88-54.jpg
1,/kaggle/input/pets-facial-expression-dataset/Other/45.jpg
2,/kaggle/input/pets-facial-expression-dataset/Other/56.jpg
3,/kaggle/input/pets-facial-expression-dataset/Other/aug-141-02.jpg
4,/kaggle/input/pets-facial-expression-dataset/Other/20.jpg


In [7]:
# Get the parent folder name of each image path
image_paths = meta_df["image"].tolist()
parent_folder_names = []
for image_path in image_paths:
    parent_folder_name = os.path.dirname(image_path)
    label_value = parent_folder_name.split("/")[-1]
    parent_folder_names.append(label_value)

# Add the "label" column to the DataFrame
meta_df["label"] = parent_folder_names

# Print the DataFrame
meta_df.head()

,image,label
0,/kaggle/input/pets-facial-expression-dataset/Other/aug-88-54.jpg,Other
1,/kaggle/input/pets-facial-expression-dataset/Other/45.jpg,Other
2,/kaggle/input/pets-facial-expression-dataset/Other/56.jpg,Other
3,/kaggle/input/pets-facial-expression-dataset/Other/aug-141-02.jpg,Other
4,/kaggle/input/pets-facial-expression-dataset/Other/20.jpg,Other


In [8]:
meta_df['label'].value_counts()

happy    522
Sad      521
Angry    519
Other    512
Name: label, dtype: int64

In [9]:
PILImage.create('/kaggle/input/pets-facial-expression-dataset/Angry/aug-155-08.jpg').shape

(179, 179)

## Crop with padding option to avoid check for image size

In [10]:
meta_df.iloc[11, 0]

Path('/kaggle/input/pets-facial-expression-dataset/Other/aug-152-19.jpg')

In [11]:
%%time
# image_df.loc[:, 'shape'] = image_df['image'].apply(lambda x: PILImage.create(x).shape)
shape_lst = []
for i in range(len(meta_df['image'])):
    shp = PILImage.create(meta_df.iloc[i, 0]).shape
    shape_lst.append(shp)

CPU times: user 2.69 s, sys: 517 ms, total: 3.2 s
Wall time: 18.1 s


In [12]:
meta_df['shape'] = shape_lst

In [13]:
meta_df.head()

,image,label,shape
0,/kaggle/input/pets-facial-expression-dataset/Other/aug-88-54.jpg,Other,"(224, 224)"
1,/kaggle/input/pets-facial-expression-dataset/Other/45.jpg,Other,"(224, 224)"
2,/kaggle/input/pets-facial-expression-dataset/Other/56.jpg,Other,"(224, 224)"
3,/kaggle/input/pets-facial-expression-dataset/Other/aug-141-02.jpg,Other,"(224, 224)"
4,/kaggle/input/pets-facial-expression-dataset/Other/20.jpg,Other,"(224, 224)"


## Perform ML